# 1. Importing Required Modules

In [3]:
from zipfile import ZipFile 
import pandas as pd
import os

# 2. Extract data file from zip files

## 2.a. Securities master file 

In [5]:
# Listing all zip files in the directory
securities_mf_path = 'Dataset_Masterfile_Efek'
securities_mf_files = os.listdir(securities_mf_path)
securities_mf_files

['StatisEfek20210531.txt.zip',
 'StatisEfek20210630.txt.zip',
 'StatisEfek20210730.txt.zip',
 'StatisEfek20210831.txt.zip',
 'StatisEfek20210930.txt.zip',
 'StatisEfek20211029.txt.zip',
 'StatisEfek20211130.txt.zip',
 'StatisEfek20211230.txt.zip',
 'StatisEfek20220131.txt.zip',
 'StatisEfek20220225.txt.zip',
 'StatisEfek20220331.txt.zip',
 'StatisEfek20220428.txt.zip',
 'StatisEfek20220531.txt.zip',
 'StatisEfek20220630.txt.zip',
 'StatisEfek20220729.txt.zip',
 'StatisEfek20220831.txt.zip',
 'StatisEfek20220930.txt.zip',
 'StatisEfek20221031.txt.zip',
 'StatisEfek20221130.txt.zip',
 'StatisEfek20221230.txt.zip',
 'StatisEfek20230131.txt.zip',
 'StatisEfek20230228.txt.zip',
 'StatisEfek20230331.txt.zip',
 'StatisEfek20230428.txt.zip',
 'StatisEfek20230531.txt.zip',
 'StatisEfek20230627.txt.zip',
 'StatisEfek20230731.txt.zip',
 'StatisEfek20230831.txt.zip',
 'StatisEfek20230929.txt.zip',
 'StatisEfek20231031.txt.zip',
 'StatisEfek20231130.txt.zip',
 'StatisEfek20231229.txt.zip',
 'Statis

## 2.b. Securities ownership master file

In [6]:
# Listing all zip files in the directory
securities_os_path = 'Dataset_Ownership_Efek'
securities_os_files = os.listdir(securities_os_path)
securities_os_files

['BalanceposEfek20210531.zip',
 'BalanceposEfek20210630.zip',
 'BalanceposEfek20210730.zip',
 'BalanceposEfek20210831.zip',
 'BalanceposEfek20210930.zip',
 'BalanceposEfek20211029.zip',
 'BalanceposEfek20211130.zip',
 'BalanceposEfek20211230.zip',
 'BalanceposEfek20220131.zip',
 'BalanceposEfek20220225.zip',
 'BalanceposEfek20220331.zip',
 'BalanceposEfek20220428.zip',
 'BalanceposEfek20220531.zip',
 'BalanceposEfek20220630.zip',
 'BalanceposEfek20220729.zip',
 'BalanceposEfek20220831.zip',
 'BalanceposEfek20220930.zip',
 'BalanceposEfek20221031.zip',
 'BalanceposEfek20221130.zip',
 'BalanceposEfek20221230.zip',
 'BalanceposEfek20230131.zip',
 'BalanceposEfek20230228.zip',
 'BalanceposEfek20230331.zip',
 'BalanceposEfek20230428.zip',
 'BalanceposEfek20230531.zip',
 'BalanceposEfek20230627.zip',
 'BalanceposEfek20230731.zip',
 'BalanceposEfek20230831.zip',
 'BalanceposEfek20230929.zip',
 'BalanceposEfek20231031.zip',
 'BalanceposEfek20231130.zip',
 'BalanceposEfek20231229.zip',
 'Balanc